# Import Libraries

In [59]:
#Importing libraries needed
import pandas as pd
import numpy as np
from collections import Counter
import re 
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Open Dataset

In [60]:
df = pd.read_csv(r'C:\Users\salom\OneDrive\Área de Trabalho\DSFinal\IMDB Horror movies.csv')

In [61]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",NaN
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,NaN,"$30,000"
2,Sleepwalking (2017),Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...",Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",NaN
3,Treasure Chest of Horrors II (2013),Comedy| Horror| Thriller,23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...",Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",NaN
4,Infidus (2015),Crime| Drama| Horror,10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...",Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,NaN,NaN


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              3328 non-null   object 
 1   Genres             3327 non-null   object 
 2   Release Date       3328 non-null   object 
 3   Release Country    3328 non-null   object 
 4   Movie Rating       1451 non-null   object 
 5   Review Rating      3076 non-null   float64
 6   Movie Run Time     2784 non-null   object 
 7   Plot               3327 non-null   object 
 8   Cast               3314 non-null   object 
 9   Language           3257 non-null   object 
 10  Filming Locations  2096 non-null   object 
 11  Budget             1234 non-null   object 
dtypes: float64(1), object(11)
memory usage: 312.1+ KB


# Get Cast Frequency

In [63]:
# Split the cast column by the '|' character and flatten the result into a list
cast_list = df['Cast'].str.split('|').explode()

# Count the frequency of each cast member
cast_counts = Counter(cast_list)

# Convert the cast counts to a DataFrame
cast_counts_df = pd.DataFrame(list(cast_counts.items()), columns=['Cast Member', 'Frequency'])

# Sort by frequency in descending order
cast_counts_df = cast_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
cast_counts_df = cast_counts_df.reset_index(drop=True)

print(cast_counts_df)

                 Cast Member  Frequency
0              Lloyd Kaufman         29
1               Eric Roberts         28
2              Debbie Rochon         23
3                Maria Olsen         21
4             Brinke Stevens         18
...                      ...        ...
33877  Kathy Blaze Jefferson          1
33878            Liam Hawley          1
33879         Katie Hargrave          1
33880            Ariel Gomez          1
33881        Gabriel Daniels          1

[33882 rows x 2 columns]


In [64]:
df['Cast Member'] = cast_counts_df['Cast Member'] 
df['Cast Member Frequency'] = cast_counts_df['Frequency']

# Director

In [65]:
# Extract the "Directed by" part and director's name using a regular expression
df['Director'] = df['Plot'].str.extract(r'Directed by (.+?)\.')

# Remove the "Directed by" part from the original column
df['Description'] = df['Plot'].str.replace(r'Directed by .+?\.', '')

df.head()

C:\Users\salom\AppData\Local\Temp\ipykernel_1932\1749570473.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Description'] = df['Plot'].str.replace(r'Directed by .+?\.', '')


,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",NaN,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo..."
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,NaN,"$30,000",Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car..."
2,Sleepwalking (2017),Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...",Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",NaN,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl..."
3,Treasure Chest of Horrors II (2013),Comedy| Horror| Thriller,23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...",Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",NaN,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ..."
4,Infidus (2015),Crime| Drama| Horror,10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...",Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,NaN,NaN,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio..."


# Title Count 

In [66]:


def remove_less_informative(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    
    # Define parts of speech to be removed
    less_informative_pos = set(['DT', 'CC', 'IN', 'PRP', 'MD', 'CD'])
    
    # Define punctuation to be removed
    punctuation = set(string.punctuation)
    
    # Filter words based on parts of speech and punctuation
    filtered_words = [word for word, pos in tagged_words if pos not in less_informative_pos and word not in punctuation]
    return ' '.join(filtered_words)

# Apply the function to the DataFrame
df['Title'] = df['Title'].apply(remove_less_informative)

In [67]:
# Tokenize the titles into words (split by whitespace)
words = ' '.join(df['Title']).split()

# Remove punctuation and convert to lowercase (optional)
words = [re.sub(r'[^\w\s]', '', word).lower() for word in words]

# Count the frequency of each word
word_counts = Counter(words)

# Convert the word counts to a DataFrame
title_counts_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Frequency'])

# Sort by frequency in descending order
title_counts_df = title_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
title_counts_df = title_counts_df.reset_index(drop=True)

title_counts_df.head()

,Word,Frequency
0,dead,111
1,s,103
2,house,71
3,night,60
4,blood,60


In [68]:
# add to main dataframe 
df['Title Word'] = title_counts_df['Word'] 
df['Title Word Frequency'] = title_counts_df['Frequency']

In [69]:
# Replace "|" with ", " in the "Cast" column
df['Cast'] = df['Cast'].str.replace('|', ', ')

C:\Users\salom\AppData\Local\Temp\ipykernel_1932\865710607.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Cast'] = df['Cast'].str.replace('|', ', ')


In [70]:
# Replace "|" with ", " in the "Cast" column
df['Genres'] = df['Genres'].str.replace('|', ', ')

C:\Users\salom\AppData\Local\Temp\ipykernel_1932\1158279101.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Genres'] = df['Genres'].str.replace('|', ', ')


In [71]:
# Convert all columns with object datatype to string
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  3328 non-null   object 
 1   Genres                 3328 non-null   object 
 2   Release Date           3328 non-null   object 
 3   Release Country        3328 non-null   object 
 4   Movie Rating           3328 non-null   object 
 5   Review Rating          3076 non-null   float64
 6   Movie Run Time         3328 non-null   object 
 7   Plot                   3328 non-null   object 
 8   Cast                   3328 non-null   object 
 9   Language               3328 non-null   object 
 10  Filming Locations      3328 non-null   object 
 11  Budget                 3328 non-null   object 
 12  Cast Member            3328 non-null   object 
 13  Cast Member Frequency  3328 non-null   int64  
 14  Director               3328 non-null   object 
 15  Desc

# Get Word Freqeuncy of Plot

In [73]:
# Tokenize the text in the "Description" column
words = ' '.join(df['Description']).split()

# Assuming you have a DataFrame called "news" with a "headline" column
# Define a list of stopwords
stop_words = set(stopwords.words('english'))

# Remove punctuation and convert to lowercase (optional)
words = [re.sub(r'[^\w\s]', '', word).lower() for word in words]

# Count the frequency of each word
word_counts = Counter(words)

# Convert the word counts to a DataFrame
word_counts_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Frequency'])

# Sort by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
word_counts_df = word_counts_df.reset_index(drop=True)

word_counts_df.head(100)

,Word,Frequency
0,a,6542
1,the,6103
2,with,4310
3,to,3602
4,of,3538
...,...,...
95,being,131
96,there,130
97,strange,130
98,dead,129


In [74]:
df['Description Word'] = word_counts_df['Word'] 
df['Description Word Frequency'] = word_counts_df['Frequency']
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",nan,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,nan,"$30,000",Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",nan,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",nan,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,nan,nan,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538


In [58]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


# Cleaning Budget

In [18]:
# Remove non-numeric characters from the "Budget" column
df['Budget'] = df['Budget'].str.replace(r'[^0-9.]', '', regex=True)

df

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,nan,30000,Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,nan,,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,Victor Frankenstein,"Drama, Horror, Sci-Fi, Thriller",25-Nov-15,USA,PG-13,6.0,110 min,Directed by Paul McGuigan. With Daniel Radclif...,"Daniel Radcliffe, Jessica Brown Findlay, Brons...",English,"London, England, UK",40000000,Alexis Kendra,2,Paul McGuigan,"With Daniel Radcliffe, James McAvoy, Jessica ...",nan,NaN,gomez,5
3324,Exorcism Molly Hartley,Horror,9-Oct-15,USA,UNRATED,4.6,96 min,"Directed by Steven R. Monroe. With Sarah Lind,...","Sarah Lind, Devon Sawa, Gina Holden, Peter Mac...",English,"Winnipeg, Manitoba, Canada",,Scott Gillespie,2,Steven R,"Monroe. With Sarah Lind, Devon Sawa, Gina Hol...",nan,NaN,enemies,5
3325,Talon Falls,Horror,13-Oct-17,USA,nan,4.6,75 min,"Directed by Joshua Shreve. With Brad Bell, Jor...","Brad Bell, Jordyn Rudolph, Ryan Rudolph, Morga...",English,nan,,Bailey Gaddis,2,Joshua Shreve,"With Brad Bell, Jordyn Rudolph, Ryan Rudolph,...",nan,NaN,fateful,5
3326,BloodRayne Third Reich,"Action, Adventure, Fantasy, Horror",17-Mar-12,Japan,R,3.0,79 min,"Directed by Uwe Boll. With Natassia Malthe, Br...","Natassia Malthe, Brendan Fletcher, Michael Par...",English,"Zagreb, Croatia",10000000,Anastasia Baranova,2,Uwe Boll,"With Natassia Malthe, Brendan Fletcher, Micha...",nan,NaN,solitary,5


In [19]:
df.to_csv('CleanHorror.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'CleanHorror.csv'

In [23]:
"df = pd.read_csv(r'C:\Users\salom\OneDrive\Área de Trabalho\DSFinal\Horror.csv')"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 21-22: truncated \UXXXXXXXX escape (2946583026.py, line 1)

In [24]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,nan,30000,Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,nan,,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538


In [25]:
# Split the "Genres" column by commas and stack the data
stacked_data = df['Genres'].str.split(', ').explode()

# Count the frequency of each word
word_counts = stacked_data.value_counts()

# Create a DataFrame with the word frequencies
word_counts_df = pd.DataFrame({'Word': word_counts.index, 'Frequency': word_counts.values})

word_counts_df

,Word,Frequency
0,Horror,3327
1,Thriller,1378
2,Drama,531
3,Comedy,513
4,Mystery,453
5,Action,336
6,Sci-Fi,309
7,Fantasy,229
8,Crime,121
9,Adventure,116


In [26]:
df['Genres Word'] = word_counts_df['Word']
df['Genres Frequency'] = word_counts_df['Frequency']

In [27]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,...,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency,Genres Word,Genres Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,...,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542,Horror,3327.0
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,...,Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103,Thriller,1378.0
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,...,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310,Drama,531.0
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,...,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602,Comedy,513.0
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,...,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538,Mystery,453.0


In [28]:
df.to_csv('Horror_copy.csv', index=False)

# Looking at headlines from Huffpost from 2014 to 2017

In [57]:
import pandas as pd

file_path = r'C:\Users\salom\OneDrive\Área de Trabalho\DSFinal\News_Category_Dataset_v3.json'

# Read the JSON file with lines=True to handle multiple JSON objects
df = pd.read_json(file_path, lines=True)

# Filter the DataFrame to include only rows where "category" is "U.S. NEWS"
us_news_df = df[df["category"] == "POLITICS"]

# Select the "short_description" column from the filtered DataFrame
us_news_short_desc = us_news_df["headline"]


In [30]:
# Create a DataFrame from the Series
us_news_short_desc = pd.DataFrame({"headline": us_news_short_desc})

In [31]:
us_news_short_desc.head()

,headline
21,Biden Says U.S. Forces Would Defend Taiwan If ...
24,‘Beautiful And Sad At The Same Time’: Ukrainia...
30,Biden Says Queen's Death Left 'Giant Hole' For...
40,Bill To Help Afghans Who Escaped Taliban Faces...
44,Mark Meadows Complies With Justice Dept. Subpo...


In [32]:
headline = us_news_short_desc

In [33]:
headline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35602 entries, 21 to 133657
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  35602 non-null  object
dtypes: object(1)
memory usage: 556.3+ KB


In [34]:
# Reset the index and move it to a column
headline = headline.reset_index()

# Rename the column to "ID"
headline = headline.rename(columns={'index': 'ID'})

In [35]:
headline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35602 entries, 0 to 35601
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        35602 non-null  int64 
 1   headline  35602 non-null  object
dtypes: int64(1), object(1)
memory usage: 556.4+ KB


In [36]:
result = headline.loc[headline['ID'] == 21497]
print(result)

         ID                                           headline
7461  21497  Trump Says He'd 'Certainly Consider' Sending N...


In [37]:
# Delete rows before index 7461
headline = headline.drop(headline.index[headline.index < 47146])

In [38]:
headline.head()

,ID,headline
7461,21497,Trump Says He'd 'Certainly Consider' Sending N...
7462,21502,ACLU Sues Over 10-Year-Old Undocumented Immigr...
7463,21503,Why Trump’s Pick For Chair Of The Federal Rese...
7464,21505,Donald Trump Dismisses Campaign Adviser As 'Yo...
7465,21513,"I Worked For Bill Clinton In The '90s. Today, ..."


In [39]:
news = headline 

In [40]:
news.head()

,ID,headline
7461,21497,Trump Says He'd 'Certainly Consider' Sending N...
7462,21502,ACLU Sues Over 10-Year-Old Undocumented Immigr...
7463,21503,Why Trump’s Pick For Chair Of The Federal Rese...
7464,21505,Donald Trump Dismisses Campaign Adviser As 'Yo...
7465,21513,"I Worked For Bill Clinton In The '90s. Today, ..."


In [41]:

# Assuming you have a DataFrame called "news" with a "headline" column
# Define a list of stopwords
stop_words = set(stopwords.words('english'))

# Tokenize the "headline" column
news["headline_tokens"] = news["headline"].apply(word_tokenize)

# Filter out the stopwords
news["filtered_tokens"] = news["headline_tokens"].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

# Flatten the list of filtered tokens
filtered_tokens_flat = [word for tokens in news["filtered_tokens"] for word in tokens]

# Count the frequency of each word
word_counts = Counter(filtered_tokens_flat)

# Convert the word counts to a DataFrame
word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index', columns=['Frequency'])

# Sort the DataFrame by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# Now, word_counts_df contains the frequency of each word in the "headline" column
# excluding common prepositions and stopwords

In [43]:
word_counts_df.reset_index()

,index,Frequency
0,Trump,7175
1,'s,5694
2,",",4758
3,',4108
4,:,4091
...,...,...
20099,C.,1
20100,Bolduc,1
20101,Pu,1
20102,Snatchers,1


# Linechart

Two axis: two different scales of y-axis (frequency of movies and frequency of headlines), might have to experiment and see if you go with percent and that would be one y-axis. 

So if entire headline mentions a theme than associate it with that theme. 

X axis will be year 

Code your words as a list and then use list comprehensive 

"filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]"

turn list into set 

write three functions because you'll create three columns

https://www.w3schools.com/python/ref_set_intersection.asp

violence, other, family 

In [45]:
violence = ['Assault',
'Homicide',
'Shooting',
'Stabbing',
'Robbery',
'Murder',
'Crime',
'Gunfight',
'Terrorism',
'Riot',
'Conflict',
'Hostage',
'Violence',
'Bloodshed',
'Brutality',
'War',
'Gang',
'Attack',
'Kidnapping',
'Domestic abuse',
'Massacre',
'Coup',
'Aggression',
'Armageddon',
'Revolt',
'Slaughter',
'Firearm',
'Genocide',
'Mayhem',
'Warfare',
'Shooting spree',
'Street fight',
'Crisis',
'Shootout',
'Insurgency',
'Bullying',
'Weapon',
'Conflict resolution',
'Assault and battery',
'Fighting',
'Violent crime',
'Armed conflict',
'Clash',
'Brawl',
'Hostility',
'Torture',
'Assault rifle',
'Assassination',
'Gang violence',
'Unrest',
'Warfare',
'Mass shooting',
'Gun violence',
'Civil unrest',
'Vandalism',
'Confrontation',
'Hate crime',
'Domestic violence',
'Criminal activity',
'Retaliation',
'Gun control',
'War zone',
'Crime scene',
'Paramilitary',
'Extremism',
'Fear',
'Agitator',
'Home invasion',
'Armament',
'Violent protest',
'Terrorism alert',
'Fighting back',
'Aggressive behavior',
'Gang war',
'Hate group',
'Weaponry',
'Siege',
'Agitator',
'Violent offender',
'Sniper',
'Police brutality',
'Conflict zone',
'Hate speech',
'Standoff',
'Agitprop',
'Knife crime',
'Civil disturbance',
'Rioting',
'Gunshot wound',
'Revenge',
'Domestic dispute',
'Paramilitary group',
'Aggressive rhetoric',
'Gang activity',
'Hostage crisis',
'Violent incident',
'War crimes',
'Hate crime legislation',
'Armed confrontation',
'Terrorism threat']

print(violence)

['Assault', 'Homicide', 'Shooting', 'Stabbing', 'Robbery', 'Murder', 'Crime', 'Gunfight', 'Terrorism', 'Riot', 'Conflict', 'Hostage', 'Violence', 'Bloodshed', 'Brutality', 'War', 'Gang', 'Attack', 'Kidnapping', 'Domestic abuse', 'Massacre', 'Coup', 'Aggression', 'Armageddon', 'Revolt', 'Slaughter', 'Firearm', 'Genocide', 'Mayhem', 'Warfare', 'Shooting spree', 'Street fight', 'Crisis', 'Shootout', 'Insurgency', 'Bullying', 'Weapon', 'Conflict resolution', 'Assault and battery', 'Fighting', 'Violent crime', 'Armed conflict', 'Clash', 'Brawl', 'Hostility', 'Torture', 'Assault rifle', 'Assassination', 'Gang violence', 'Unrest', 'Warfare', 'Mass shooting', 'Gun violence', 'Civil unrest', 'Vandalism', 'Confrontation', 'Hate crime', 'Domestic violence', 'Criminal activity', 'Retaliation', 'Gun control', 'War zone', 'Crime scene', 'Paramilitary', 'Extremism', 'Fear', 'Agitator', 'Home invasion', 'Armament', 'Violent protest', 'Terrorism alert', 'Fighting back', 'Aggressive behavior', 'Gang war

In [46]:
other = ['Diversity',
'Inclusion',
'Integration',
'Acceptance',
'Immigration',
'Multiculturalism',
'Tolerance',
'Refugees',
'Assimilation',
'Migrants',
'Foreigners',
'Xenophobia',
'Pluralism',
'Cultural Exchange',
'Cross-Cultural',
'Migrant Workers',
'Naturalization',
'Asylum Seekers',
'Global Citizens',
'Ethnicity',
'Resettlement',
'Humanitarian Aid',
'Citizenship',
'Integration Policies',
'Cultural Diversity',
'Global Migration',
'Immigrant Rights',
'Foreign Born',
'Immigrant Communities',
'Diversity Training',
'Anti-Discrimination',
'Cultural Enrichment',
'Diaspora',
'Cross-Border',
'Migration Patterns',
'Cultural Sensitivity',
'Refugee Crisis',
'Diversity and Inclusion',
'Migrant Integration',
'Cultural Assimilation',
'Xenophobic Attitudes',
'Refugee Resettlement',
'Immigrant Advocacy',
'Diversity Initiatives',
'Global Mobility',
'Cultural Exchange Programs',
'Migration Challenges',
'Immigrant Education',
'Foreign Nationals',
'Xenophobic Behavior',
'Cultural Adaptation',
'Immigrant Rights',
'Asylum Policies',
'Migrant Labor',
'Global Citizenship',
'Cultural Identity',
'Integration Strategies',
'Diversity Awareness',
'Migrant Flows',
'Foreign-Born Populations',
'Immigrant Contributions',
'Xenophobia Prevention',
'Cultural Pluralism',
'Refugee Support',
'Immigrant Inclusion',
'Diversity Workshops',
'Cultural Harmony',
'Immigrant Resilience',
'Cross-Cultural Understanding',
'Migrant Rights',
'Ethnic Communities',
'Refugee Protection',
'Cultural Enrichment Programs',
'Xenophobia Awareness',
'Diaspora Connections',
'Migration Policies',
'Immigrant Empowerment',
'Cultural Integration',
'Global Migrant Trends',
'Anti-Xenophobia',
'Refugee Integration',
'Diversity Celebrations',
'Migrant Advocacy',
'Cultural Sensitivity Training',
'Xenophobic Incidents',
'Immigrant Success Stories',
'Diverse Society',
'Cultural Enrichment Initiatives',
'Immigrant Inclusivity',
'Cross-Border Cooperation',
'Migration Solutions',
'Xenophobia Eradication',
'Refugee Assistance',
'Cultural Exchange Initiatives',
'Diaspora Engagement',
'Migrant Support Programs',
'Cultural Exchange Experiences',
'Immigrant Solidarity',
'Cultural Acceptance',
'Global Migration Issues']
print(other)

['Diversity', 'Inclusion', 'Integration', 'Acceptance', 'Immigration', 'Multiculturalism', 'Tolerance', 'Refugees', 'Assimilation', 'Migrants', 'Foreigners', 'Xenophobia', 'Pluralism', 'Cultural Exchange', 'Cross-Cultural', 'Migrant Workers', 'Naturalization', 'Asylum Seekers', 'Global Citizens', 'Ethnicity', 'Resettlement', 'Humanitarian Aid', 'Citizenship', 'Integration Policies', 'Cultural Diversity', 'Global Migration', 'Immigrant Rights', 'Foreign Born', 'Immigrant Communities', 'Diversity Training', 'Anti-Discrimination', 'Cultural Enrichment', 'Diaspora', 'Cross-Border', 'Migration Patterns', 'Cultural Sensitivity', 'Refugee Crisis', 'Diversity and Inclusion', 'Migrant Integration', 'Cultural Assimilation', 'Xenophobic Attitudes', 'Refugee Resettlement', 'Immigrant Advocacy', 'Diversity Initiatives', 'Global Mobility', 'Cultural Exchange Programs', 'Migration Challenges', 'Immigrant Education', 'Foreign Nationals', 'Xenophobic Behavior', 'Cultural Adaptation', 'Immigrant Rights'

In [47]:
family = ['Love',
'Relationships',
'Parenting',
'Children',
'Marriage',
'Siblings',
'Home',
'Caring',
'Support',
'Bonds',
'Generations',
'Nurturing',
'Unity',
'Traditions',
'Families',
'Communication',
'Togetherness',
'Adoption',
'Parent-Child',
'Spouse',
'Kinship',
'Grandparents',
'Family Dynamics',
'Cherish',
'Heritage',
'Affection',
'Household',
'Family Values',
'Childhood',
'Relatives',
'Foster Care',
'Family Time',
'Blended Family',
'Cohesion',
'Extended Family',
'Family Traditions',
'Family Bonding',
'Intimacy',
'Domestic Bliss',
'Support System',
'Sisterhood',
'Brotherhood',
'Parenthood',
'Family Planning',
'Family Ties',
'In-Laws',
'Maternal Love',
'Paternal Love',
'Grandchildren',
'Child Rearing',
'Familial Relations',
'Custody',
'Family Celebrations',
'Emotional Well-being',
'Family Traditions',
'Elder Care',
'Family Reunions',
'Nuclear Family',
'Parental Guidance',
'Fostering',
'Quality Time',
'Family Support',
'Family Structure',
'Family Unity',
'Family Resilience',
'Adoptive Parents',
'Filial Piety',
'Household Harmony',
'Family Safety',
'Sisterly Love',
'Brotherly Love',
'Family Milestones',
'Family Wellness',
'Cousins',
'Family Bonds',
'Multi-Generational',
'Inheritance',
'Sororal Love',
'Fraternal Love',
'Family Affection',
'Maternal Care',
'Paternal Care',
'Family Unity',
'Familial Love',
'Family Conflict',
'Generational Gap',
'Elderly Care',
'Parental Roles',
'Sibling Rivalry',
'Family Responsibilities',
'Child Development',
'Custodial Arrangements',
'Familial Support',
'Intergenerational Relationships',
'Family Values',
'Maternal Instinct',
'Paternal Instinct',
'Foster Parenting',
'Family Happiness',
'Familial Well-being']
print(family)

['Love', 'Relationships', 'Parenting', 'Children', 'Marriage', 'Siblings', 'Home', 'Caring', 'Support', 'Bonds', 'Generations', 'Nurturing', 'Unity', 'Traditions', 'Families', 'Communication', 'Togetherness', 'Adoption', 'Parent-Child', 'Spouse', 'Kinship', 'Grandparents', 'Family Dynamics', 'Cherish', 'Heritage', 'Affection', 'Household', 'Family Values', 'Childhood', 'Relatives', 'Foster Care', 'Family Time', 'Blended Family', 'Cohesion', 'Extended Family', 'Family Traditions', 'Family Bonding', 'Intimacy', 'Domestic Bliss', 'Support System', 'Sisterhood', 'Brotherhood', 'Parenthood', 'Family Planning', 'Family Ties', 'In-Laws', 'Maternal Love', 'Paternal Love', 'Grandchildren', 'Child Rearing', 'Familial Relations', 'Custody', 'Family Celebrations', 'Emotional Well-being', 'Family Traditions', 'Elder Care', 'Family Reunions', 'Nuclear Family', 'Parental Guidance', 'Fostering', 'Quality Time', 'Family Support', 'Family Structure', 'Family Unity', 'Family Resilience', 'Adoptive Parent

# Headline 2017

In [50]:
headline_2017 = headline['headline']
headline_2017.head()

7461    Trump Says He'd 'Certainly Consider' Sending N...
7462    ACLU Sues Over 10-Year-Old Undocumented Immigr...
7463    Why Trump’s Pick For Chair Of The Federal Rese...
7464    Donald Trump Dismisses Campaign Adviser As 'Yo...
7465    I Worked For Bill Clinton In The '90s. Today, ...
Name: headline, dtype: object